<a href="https://colab.research.google.com/github/tanmayispiddy07/gemini_phrase_recognization/blob/main/phase_generator_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

this is used to add the genai authorisation to our notebook

In [ ]:
import os
if 'COLAB_RELEASE_TAG' in os.environ:
  from google.colab import userdata
  import pathlib
  pathlib.Path('/content/client_secret.json').write_text(userdata.get('CLIENT_SECRET'))

  # Use `--no-browser` in colab
  !gcloud auth application-default login --no-browser --client-id-file client_secret.json --scopes='https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/generative-language.tuning'
else:
  !gcloud auth application-default login --client-id-file client_secret.json --scopes='https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/generative-language.tuning'

You are authorizing client libraries without access to a web browser. Please run the following command on a machine with a web browser and copy its output back here. Make sure the installed gcloud version is 372.0.0 or newer.

gcloud auth application-default login --remote-bootstrap="https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=9418359443-nldovmhh4iq7db59e4311n94tounuk1g.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgenerative-language.tuning&state=bsphrQrnmomjNOw4GsBpZfOAQJe2ad&access_type=offline&code_challenge=S7eO4W0OR_v0d-Qn-Wbhc4wSHNmr3mFarHA151O_QkM&code_challenge_method=S256&token_usage=remote"


Enter the output of the above command: https://localhost:8085/?state=bsphrQrnmomjNOw4GsBpZfOAQJe2ad&code=4/0AQlEd8ypboJcwFvn5DItNHHgZzBIT76ipcO4lh1MddyTR9aEbIqvL_RJjegBMjC29in8OA&scope=https://www.googleapis.com/auth/cloud-platform%20https://www.googleapis.com/auth/generative

INSTALL AND IMPORT THE GOOGLE GEN AI TO UTLIZE THE GEMINI AND GEMMA MODELS

In [ ]:
!pip install -q google-generativeai

In [ ]:
import google.generativeai as genai

THIS CODE IS USE TO RETRIVE THE LIST OF THE MODELS IN YOUR GOOGLE CLOUD

In [ ]:
for i, m in zip(range(20), genai.list_models()):
  print(m.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-8b-exp-0827


BEFORE CREATING A TUNED MODEL WE MUST SELECT THE BASE MODEL TO TUNE IT

In [ ]:
base_model = [
    m for m in genai.list_models()
    if "createTunedModel" in m.supported_generation_methods][1]
base_model

Model(name='models/gemini-1.5-flash-001-tuning',
      base_model_id='',
      version='001',
      display_name='Gemini 1.5 Flash 001 Tuning',
      description='Fast and versatile multimodal model for scaling across diverse tasks',
      input_token_limit=16384,
      output_token_limit=8192,
      supported_generation_methods=['generateContent', 'countTokens', 'createTunedModel'],
      temperature=1.0,
      max_temperature=2.0,
      top_p=0.95,
      top_k=64)

LOADING THE TRAIN DATA

In [ ]:
df = pd.read_csv('/content/train_random_632.csv', encoding ='latin-1')

RANDOMIZING THE TRAIN DATESET WHICH WILLL BE USED FOR EFFICENT TRANING

In [ ]:
df_shuffled.head()

,Tweet,Target,Stance,ID,Target_ID,Dataset_id
0,but i believe that the government will put it ...,nuclear energy,NONE,13,14,1
1,everyone looks the same dreary and vulgar,school uniforms,AGAINST,14,15,1
2,put plainly if somebody believes that they are...,gun control,FAVOR,10,11,1
3,people that are surprising trump fiorina why c...,Hillary Clinton,NONE,2,2,0
4,in christ we are righteous and in practice we ...,Atheism,AGAINST,0,0,0


In [ ]:
df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)

DEFINING THE INPUT TEXT AND OUTPUT DATA TO TRAIN

In [ ]:
df_sampled = pd.DataFrame()
df_sampled['text_input']= df_shuffled['Tweet'].head(150)
df_sampled['output'] = df_shuffled['Target'].head(150)


DEFINING AND HYPERTUNING THE MODEL BY BASE MODEL

In [ ]:
name = f'phase-generator-11'
operation = genai.create_tuned_model(

    source_model=base_model.name,
    training_data=df_sampled,
    id = name,
    epoch_count = 2,
    batch_size=5,
    learning_rate=0.0001,
)

USED TO RETRIVE THE TRAIN STATUS

In [ ]:
import time

for status in operation.wait_bar():
  time.sleep(30)

  0%|          | 0/60 [00:00<?, ?it/s]

KeyboardInterrupt: 

DEFINING THE MODEL

In [ ]:
model = genai.get_model(f'models/gemini-1.5-flash-001')

model

Model(name='models/gemini-1.5-flash-001',
      base_model_id='',
      version='001',
      display_name='Gemini 1.5 Flash 001',
      description='Fast and versatile multimodal model for scaling across diverse tasks',
      input_token_limit=1048576,
      output_token_limit=8192,
      supported_generation_methods=['generateContent', 'countTokens', 'createCachedContent'],
      temperature=1.0,
      max_temperature=2.0,
      top_p=0.95,
      top_k=64)

In [ ]:
import pandas as pd
import seaborn as sns

model = operation.result()

snapshots = pd.DataFrame(model.tuning_task.snapshots)

sns.lineplot(data=snapshots, x = 'epoch', y='mean_loss')

KeyboardInterrupt: 

IMPORTING THE HARMCATEGORY AND BLOCK TO AVOID THE RESTRICTING WILL GENERATING

In [ ]:
import pandas as pd
from google.generativeai.types import HarmCategory, HarmBlockThreshold
import json

LOADING TEST CSV

In [ ]:
df = pd.read_csv('/content/Test_dataset(unseen)_zero_shot_cleaned_data.csv' , encoding='latin-1')

In [ ]:
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",

}

model = genai.GenerativeModel(
  model_name="models/gemini-1.5-flash-001",
  generation_config=generation_config,

)

chat_session = model.start_chat(
  history=[
    {
      "role": "model",
      "parts": [
        "```json\n{\n  \"target\": \"Creationism\",\n  \"stance\": \"Negative\" \n}\n``` \n",
      ],
    },
    {
      "role": "model",
      "parts": [
        "```json\n{\n  \"target\": \"MeToo\", \n  \"stance\": \"Against\" \n}\n```",
      ],
    },

    {
      "role": "model",
      "parts": [
        "```json\n{\n  \"target\": \"ClimateChange\",\n  \"stance\": \"Favor\" \n}'''",
      ],
    },
  ]
)

DEFINING THE PROMPT AND MAKING PREDICT FUNCTION

In [ ]:
def predict_target_and_stance(tweet_text):
    json_response = chat_session.send_message(
        [tweet_text],safety_settings={
            HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        }
    )

    return json_response.to_dict()

RETRIVING THE RESPONSE FROM THE MODEL AND DOING PREPROCESSING

In [ ]:
response_json = predict_target_and_stance(df['Tweet'][2000])
json_text=response_json['candidates'][0]['content']['parts'][0]['text']
json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
print(json_text_cleaned)

{
  "target": "MeToo",
  "stance": "Neutral" 
}


In [ ]:
print(df['Tweet'][2000])
print(df['GT Stance( Ground Truth)'][2000])
print(df['GT Target(Ground Truth)'][2000])

Farah dodges questions after SajidÃÂ¢ÃÂÃÂs #MeTooÃÂÃÂ episode https://t.co/TZt7wgNWos https://t.co/YJat4pkQW1
NONE
metoo movement


In [ ]:
df.head()

,Tweet,GT Target(Ground Truth),GT Stance( Ground Truth),Mapped Target,Generated Targets,Best Gen Target,Cleaned Tweet
0,My point was not that Genesis contradicted its...,creationism,AGAINST,NaN,NaN,NaN,point genesis contradicted took literally cont...
1,I believe in evolution; evolution is the mech...,creationism,FAVOR,NaN,NaN,NaN,believe evolution evolution mechanism species ...
2,Should we just reject the big-bang as being un...,creationism,FAVOR,NaN,NaN,NaN,reject bigbang unscientific since would requir...
3,It is not 'appropriate' to teach creationism a...,creationism,AGAINST,NaN,NaN,NaN,appropriate teach creationism means upholding ...
4,Genetic tests have proven that we are very cl...,creationism,AGAINST,NaN,NaN,NaN,genetic tests proven closely related certain a...


MAKING THE PREDICTION COLUMN IN DATAFRAME AND RECORDING THE TARGETS PREDITTED FROM THE MODEL

In [ ]:
df['Predictions'] = None

for i in range(100):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [ ]:
for i in range(100,200):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [ ]:
for i in range(200,300):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [ ]:
for i in range(300,400):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [ ]:
for i in range(400,500):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [ ]:
for i in range(500,600):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [ ]:
for i in range(600,700):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [ ]:
for i in range(700,800):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [ ]:
for i in range(800,900):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [ ]:
for i in range(900,1000):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [ ]:
for i in range(1000,1100):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [ ]:
for i in range(1200,1300):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [ ]:
for i in range(1300,1400):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [ ]:
for i in range(1400,1500):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [ ]:
for i in range(1500,1600):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [ ]:
for i in range(1600,1700):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [ ]:
for i in range(1700,1800):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [ ]:
for i in range(1800,1900):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [ ]:
for i in range(1900,2000):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [ ]:
for i in range(2000,2100):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [ ]:
for i in range(2100,2200):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [ ]:
for i in range(2200,2300):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [ ]:
for i in range(2300,2400):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [ ]:
for i in range(2500,2600):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [ ]:
for i in range(2600,2700):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [ ]:
for i in range(2700,2800):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [ ]:
for i in range(2800,2900):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [ ]:
for i in range(2900,3000):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [ ]:
for i in range(3100,3200):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [ ]:
for i in range(3200,3300):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [ ]:
for i in range(3300,3400):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [ ]:
for i in range(3400,3500):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [ ]:
for i in range(3500,3600):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

PREPROCESSING OF RESULT BEFORE EVALUVATION

In [ ]:
phrases_to_remove = ['gun control']

# Remove the specified phrases from the text
for i in range(len(df)):
    p = df.loc[i, 'Predictions']
    if p is not None:  # Check if p is not None before applying replace
        for phrase in phrases_to_remove:
            p = p.replace(phrase, '').strip()
        df.at[i, 'Predictions'] = p # Update the DataFrame

In [ ]:
df['Tweet'][1034]

'as of jan the nra had approximately million members though estimates have varied from million to million members'

In [ ]:
df[30:330]

,Unnamed: 0.1,Unnamed: 0,Tweet,Target,Stance,ID,Target_ID,Predictions
320,45758,45759,whether it should be carried out or not has be...,death penalty,NONE,9,10,
321,45759,45760,no civilian s job description should include k...,death penalty,AGAINST,9,10,
322,45760,45761,granted we are not able to ask the dead whethe...,death penalty,FAVOR,9,10,death penaltydeath penalty
323,45761,45762,this system is broken,death penalty,NONE,9,10,
324,45762,45763,pro simply because an execution method may res...,death penalty,FAVOR,9,10,death penaltydeath penalty
325,45763,45764,if improvements are needed in the system of re...,death penalty,NONE,9,10,
326,45764,45765,scholars psychologists activists all pour into...,death penalty,NONE,9,10,death penaltydeath penalty
327,45765,45766,t he claim that t his sentence rests on the ir...,death penalty,NONE,9,10,
328,45766,45767,death is an experience that can not be experie...,death penalty,NONE,9,10,death penaltydeath penalty
329,45767,45768,murder always tragic seems to be a more heinou...,death penalty,NONE,9,10,gunclontrol


GIVING THE ACCURACY REPORT

In [ ]:

from sklearn.metrics import accuracy_score, classification_report

# Assuming you want to evaluate on the first 200 samples
target_accuracy = accuracy_score(df['Target'][:200], df['Predictions'][:200]) # Use 'Predictions' column from df
target_classification_report = classification_report(df['Target'][:200], df['Predictions'][:200])

print(f"Target Accuracy: {target_accuracy}")
print("Target Classification Report:")
print(target_classification_report)

Target Accuracy: 0.42
Target Classification Report:
                                                                                                                                precision    recall  f1-score   support

                                                                                                                                     0.00      0.00      0.00         0
Genetically Modified Organisms (GMOs)cloningminimum wagemarijuana legalizationabortionminimum wagemarijuana legalizationaborti       0.00      0.00      0.00         0
                                                                                                                      abortion       0.00      0.00      0.00         0
                                                                                                                       cloning       1.00      0.88      0.93        96
                                                                                                           

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [ ]:
import pandas as pd
import seaborn as sns

model = operation.result()

snapshots = pd.DataFrame(model.tuning_task.snapshots)

sns.lineplot(data=snapshots, x = 'epoch', y='mean_loss')